In [26]:
import os  # noqa: E402
os.chdir('C:/python-training/eyeglad/use_SQl')  # noqa: E402
print(os.getcwd())  # noqa: E402

C:\python-training\eyeglad\use_SQl


In [27]:
import dash
from dash import dash_table
import dash_bootstrap_components as dbc
from dash import dcc, html, Input, Output
import pandas as pd
import plotly.express as px
from use_SQL import getdata, log_in
from sqlalchemy import create_engine, inspect


In [28]:
database = 'amazon'
# 連接到 MySQL 資料庫
engine = log_in.log_in_mySQL(database)

# 建立檢查器
inspector = inspect(engine)
# 查詢所有 table
tables = inspector.get_table_names()
print("Tables:", '\n'.join(tables))

Log in successful
Tables: sponsored_brands_attributed_purchases_report
sponsored_brands_campaign_placement_report
sponsored_brands_campaign_report
sponsored_brands_category_benchmark_report
sponsored_brands_gross_and_invalid_traffic_report
sponsored_brands_keyword_placement_report
sponsored_brands_keyword_report
sponsored_brands_search_term_impression_share_report
sponsored_brands_search_term_report
sponsored_products_advertised_product_report
sponsored_products_budget_report
sponsored_products_campaign_report
sponsored_products_gross_and_invalid_traffic_report
sponsored_products_performance_over_time_report
sponsored_products_placement_report
sponsored_products_purchased_product_report
sponsored_products_search_term_impression_share_report
sponsored_products_search_term_report
sponsored_products_targeting_report


In [29]:
# 初始化Dash應用
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# 從資料庫讀取數據

table_name = "sponsored_products_targeting_report".lower()  # 替換為你的表名
df = getdata.fetch_data_from_db(database=database, table='sponsored_products_search_term_report')


Log in successful


In [30]:
# 應用佈局
app.layout = dbc.Container([
    dbc.Row(dbc.Col(html.H1("電商平台分析儀表板"), className="mb-4")),
    
    dbc.Row([
        dbc.Col([
            html.Label("選擇X軸"),
            dcc.Dropdown(
                id='x-axis',
                options=[{'label': col, 'value': col} for col in df.columns],
                value=df.columns[0],
                className="mb-2"
            ),
            html.Label("選擇Y軸"),
            dcc.Dropdown(
                id='y-axis',
                options=[{'label': col, 'value': col} for col in df.columns],
                value=df.columns[1],
                className="mb-2"
            ),
            html.Label("篩選條件"),
            dcc.Dropdown(
                id='filter-column',
                options=[{'label': col, 'value': col} for col in df.columns],
                value=df.columns[2],
                className="mb-2"
            ),
            dcc.Input(
                id='filter-value',
                placeholder='輸入篩選值...',
                type='text',
                className="mb-2"
            )
        ], width=4),
        
        dbc.Col(dcc.Graph(id='scatter-plot'), width=8)
    ]),
    
    dbc.Row(dbc.Col(html.H2("數據表"), className="mt-4")),
    
    dbc.Row(dbc.Col(dash_table.DataTable(
        id='table',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10
    )))
], fluid=True)

# 回調函數更新圖表和表格
@app.callback(
    [Output('scatter-plot', 'figure'),
     Output('table', 'data')],
    [Input('x-axis', 'value'),
     Input('y-axis', 'value'),
     Input('filter-column', 'value'),
     Input('filter-value', 'value')]
)
def update_chart(x_axis, y_axis, filter_column, filter_value):
    # 過濾數據
    if filter_value:
        filtered_df = df[df[filter_column].astype(str).str.contains(filter_value)]
    else:
        filtered_df = df

    # 創建圖表
    fig = px.scatter(filtered_df, x=x_axis, y=y_axis, color=filter_column)
    
    # 更新表格數據
    table_data = filtered_df.to_dict('records')
    
    return fig, table_data

# 運行應用
if __name__ == '__main__':
    app.run_server(debug=True)